# Creating the simple pulse sequence diagram as code

This will also for accuracy and flexibility when used for visualization

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def plt_sinc(x_start, x_dur):
    n_vals = 100
    sinc_vals = np.sinc(np.linspace(-4, 4, n_vals))
    sinc_timing = np.linspace(x_start, x_start+x_dur, n_vals)
    plt.plot(sinc_timing, sinc_vals, linewidth=0.75)

def plt_sin(x_center, x_dur, n_cycles, color):
    n_vals = 1000
    sin_vals = np.sin(n_cycles*2*np.pi*np.linspace(0,1,n_vals))/2
    sin_timing = np.linspace(x_center-x_dur/2, x_center+x_dur/2, n_vals)
    plt.plot(sin_timing, sin_vals, color, linewidth=0.75)

    
def create_simple_pulse_seq(tes, read_time=None, x_axis_duration=None):
    """
    
    """

    n_echos = len(tes)

    if read_time == None:
        read_time = tes[0]/2 - 6

    if n_echos == 1:
        colorlist = ['g']
        if x_axis_duration == None:
            x_axis_duration = tes[-1] + 2.5*read_time
    else:
        colorlist = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    if x_axis_duration == None:
        x_axis_duration = tes[-1] + 

    # Always do a 2ms sinc pulse for RF excitation (shorter isn't nicely visible)
    plt_sinc(0, 2)
    for echo_idx, echo_val in enumerate(tes):
        plt_sin(echo_val, read_time, 32, colorlist[echo_idx])
    plt.xlim(0,x_axis_duration)

create_simple_pulse_seq([13, 31.26, 49.52], None, 50)